# Data Gathering

This notebook gathers the meta-data for the evaluation. It runs the various respiratory extraction methods.

In [ ]:
import os

import respiratory_extraction.dataset as repository

dataset = repository.from_default()
subjects = dataset.get_subjects()
subjects

In [ ]:
# The scenarios in which the subjects were recorded
# scenarios = dataset.get_scenarios()
# scenarios = [
#     '101_natural_lighting',
#     # '102_artificial_lighting',
#     '103_abrupt_changing_lighting',
#     '104_dim_lighting_auto_exposure',
#     # '106_green_lighting',
#     # '107_infrared_lighting',
#     '201_shouldercheck',
#     '202_scale_movement',
#     '203_translation_movement',
#     # '204_writing'
# ]
scenarios = [
    '101_natural_lighting',
]
scenarios

In [ ]:
from datetime import datetime

hyperparameters = {
    'quality_level': 0.1,
    'quality_level_rv': 0.05,
    'filter_lowpass': 0.1,
    'filter_highpass': 0.6,
}

evaluation_metadata = {
    'start_time': datetime.now(),
    'subjects': subjects,
    'scenarios': scenarios,
    'hyperparameters': hyperparameters,
}

In [ ]:
import respiratory_extraction.utils as utils

from respiratory_extraction.models.frequency_extraction import frequency_extractor
from respiratory_extraction.models.signal_extraction import pixel_intensity, optical_flow

evaluation_entries = []
ground_truth_entries = []

faulty_scenarios = []

lowpass = hyperparameters['filter_lowpass']
highpass = hyperparameters['filter_highpass']

for subject in subjects:
    for idx, scenario in enumerate(scenarios):
        print(f'Processing {subject} - {scenario}')

        frames, params = dataset.read_video_gray(subject, scenario)

        faces = utils.detect_faces(frames[0])
        if len(faces) != 1:
            print(f'No face detected in {subject} - {scenario}')
            faulty_scenarios.append((subject, scenario))
            continue

        # Calculate the region of interest (ROI) based on the face
        roi = utils.roi_from_face(faces[0])

        # Calculate the average pixel intensity
        pi_signal = pixel_intensity.average_pixel_intensity(frames, roi=roi)
        pi_freq = frequency_extractor.with_preprocessing(
            pi_signal,
            params.fps,
            lowpass=lowpass,
            highpass=highpass,
        )

        # Calculate the optical flow
        of_signal = optical_flow.extract_signal(
            frames,
            roi=roi,
            quality_level=hyperparameters['quality_level'],
            quality_level_rv=hyperparameters['quality_level_rv'],
            use_cgof=True,
        )
        of_freq = frequency_extractor.with_preprocessing(
            of_signal,
            params.fps,
            lowpass=lowpass,
            highpass=highpass,
        )

        evaluation_entries.append({
            'subject': subject,
            'scenario': scenario,
            'method': 'pixel_intensity',
            'fft': pi_freq.fft(min_freq=lowpass, max_freq=highpass),
            'pc': pi_freq.peak_counting(),
            'cp': pi_freq.crossing_point(),
            'nfcp': pi_freq.negative_feedback_crossover_point_method(),
        })

        evaluation_entries.append({
            'subject': subject,
            'scenario': scenario,
            'method': 'optical_flow',
            'fft': of_freq.fft(min_freq=lowpass, max_freq=highpass),
            'pc': of_freq.peak_counting(),
            'cp': of_freq.crossing_point(),
            'nfcp': of_freq.negative_feedback_crossover_point_method(),
        })

        # Garbage collect the frames
        del frames

In [ ]:
faulty_scenarios

In [ ]:
evaluation_metadata['end_time'] = datetime.now()

In [ ]:
import pandas as pd

evaluation_df = pd.DataFrame(evaluation_entries)
evaluation_df.head()

In [ ]:
ground_truth_df = pd.DataFrame(ground_truth_entries)
ground_truth_df.head()

In [ ]:
evaluation_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'baseline')
if not os.path.exists(evaluation_dir):
    os.makedirs(evaluation_dir)

# Save the evaluation dataframe
csv_path = os.path.join(evaluation_dir, 'evaluation.csv')
evaluation_df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(evaluation_dir, 'hyperparameters.json')
utils.write_json(json_path, evaluation_metadata)